> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

# Exercise Set 1: Lending club

*February 5, 2019*

In this Exercise Set 1 we will investigate loan data from `Lending Club`. We will see how nested cross validation can be used  to compute a distribution of performance metrics for comparing different models. We will also try out bagging.

Below we provide some code that helps by downloading and cleaning the data. Note this takes around 1 min if you have fast internet.

In [2]:
# Note: there are three .zip files with letter = a,b,c. 
# To ensure the files download in reasonable time we 
# only work with the first of the three. If you have time
# you can modify this cell to download all three. 

import os
import requests

filenames = []
base_url = 'https://resources.lendingclub.com/'

letter = 'a'
filename = f'LoanStats3{letter}.csv.zip'
url = base_url+filename

if not os.path.exists(filename):
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)
filenames.append(filename)

The target we are going to predict is whether loan is repaid. This can be extracted from the `loan_status` below. Features names that we work with:

- **annual_inc**: The self-reported annual income provided by the borrower during registration.
- **dti**: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
- **emp_length**: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
- **grade**: LC assigned loan grade
- **home_ownership**: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
- **int_rate**: Interest Rate on the loan
- **term**: The number of payments on the loan. Values are in months and can be either 36 or 60.
- **verification_status**: Indicates if income was verified by LC, not verified, or if the income source was verified

Load and structure data

In [3]:
import numpy as np
import pandas as pd

# Read in csv files, store them
dfs = [pd.read_csv(f,header=0,skiprows=1,low_memory=False) for f in filenames]

# concatenate the dataframes (as standard there is only 1)
df = pd.concat(dfs)\
        .dropna(subset=['loan_amnt'])\
        .dropna(axis=1, how='all')

# View data
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Identify loans of interest
df = df.loc[df.loan_status.isin(['Fully Paid', 'Charged Off'])].copy()

# Clean up variables 
df['charged_off'] = (df.loan_status=='Charged Off').astype(int)
df['int_rate_f'] = df.int_rate.str[:-1].astype(float)
df['emp_length_f'] = df.emp_length\
                        .str.split(' ')\
                        .str[0].str[:2]\
                        .str.replace('<','0')\
                        .astype(float)

# label and features
y_var = 'charged_off'
X_vars = ['term', 'int_rate_f', 'grade', 'home_ownership', 'emp_length_f',
          'annual_inc', 'verification_status', 'dti']

# Create dummies
data = pd.get_dummies(df[X_vars+[y_var]], drop_first=True)\
        .dropna()\
        .reset_index(drop=True)\
        .astype(np.float64)\
        .loc[:2000]\
        .copy()

# View data
data.head()

,int_rate_f,emp_length_f,annual_inc,dti,charged_off,term_ 60 months,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
0,10.65,10.0,24000.0,27.65,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,15.27,0.0,30000.0,1.00,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,15.96,10.0,12252.0,8.72,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,13.49,10.0,49200.0,20.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,12.69,1.0,80000.0,17.94,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Now let us split the data into test and training data. To do this we use the `StratifiedShuffleSplit` from scikit learn (read more about splitting methods [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py))

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=10, test_size=.3, random_state=3)

# These are the row indices of the stratified split
data_splits = list(sss.split(data[y_var], data[y_var]))

# Separate data in y,X
y = data[y_var]
X_vars_b = data.columns!=y_var
X = data.loc[:,X_vars_b]

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, y)
model.feature_importances_

array([0.13978474, 0.13784781, 0.25765084, 0.2881301 , 0.03097463,
       0.01214228, 0.0100955 , 0.01123553, 0.01152651, 0.00931849,
       0.00072105, 0.        , 0.        , 0.01466259, 0.02718868,
       0.02759645, 0.0211248 ])

<br>
<br>

## 1.1 Model validation

We start out with some basic concepts and our goal is to estimate and evaluate the logistic regression.


> **Ex. 1.1.1:** extract the first data split and construct features, target for train, test data.
>> *Hint:* `data_splits` is a list with 10 train-test pairs of randomly selected row indices. So the k'th element of `data_splits` contains a pair (train_idx, test_idx) identifying the k'th fold in the data.

In [9]:
# [Answer to ex. 1.1.1 here]
data_splits

[(array([ 206, 1510, 1604, ...,  245,  186, 1355], dtype=int64),
  array([ 448, 1259,  869, 1467,  557, 1558, 1422, 1104, 1423,  644, 1718,
          845, 1575,  427, 1403,  661,   35,  229,  870,  295,  228, 1564,
         1272,  707, 1385, 1919,  553, 1309,  900,  373,  992,  537,  527,
           42, 1238, 1551,  212, 1701,  530,   93, 1815,  185, 1461, 1178,
          599,  123, 1652,  238, 1353,  354, 1103,  970, 1095, 1465, 1052,
         1518,  535, 1986,  615, 1731, 1816, 1447,  801,  799,  892,  366,
          893,  237,  586,  635, 1211,   16,  613,  300,  835, 1826,  275,
          499, 1444,  766, 1078,  353,    3,  505, 1611,  832,  805,  664,
          509, 1417, 1996, 1951, 1340, 1438,  310, 1077, 1527, 1057,  341,
          881,  446, 1606,  720, 1017, 1179, 1116, 1185, 1614,  128,  103,
          974, 1802, 1121, 1298, 1862,  321,  268,  469,  989,  688,  525,
          995, 1736, 1344,   44, 1312,  401,  383, 1848, 1175, 1481,  572,
          987, 1364,  671, 1970, 14

> **Ex. 1.1.2:** estimate a logistic regression model on your training data. Then compute the overall accuracy and $F_1$ score for the default class *on the training data*.
>
>> **Note**: the code below implements a pipeline which standardizes the data by converting it into zero mean and unit standard deviation. We let `C=10**10` which corresponds to no regularization. Make sure you understand why each of these steps are important by looking up the scikit learn docs.
>
>> *Hint 2:* `sklearn` has these functions built-in and are named `f1_score`, `accuracy_score`. See Raschka pp. 189-193.

In [7]:
from sklearn.preprocessing import StandardScaler        # scales variables to be mean=0,sd=1
from sklearn.linear_model import LogisticRegression     # regression model
from sklearn.pipeline import Pipeline                   # For building our model pipeline

lr = Pipeline([('scale', StandardScaler()),
               ('clf', LogisticRegression(class_weight='balanced',C=10**10, solver = 'liblinear'))])

In [8]:
# [Answer to ex. 1.1.2 here]

> **Ex. 1.1.3:** Explain some advantages of computing the $F_1$ score vs. computing the overall accuracy.

In [10]:
# [Answer to ex. 1.1.3 here]

> **Ex. 1.1.4:** Compute the *test set* accuracy and $F_1$ score. How does these compare to the results you got in exercise 1.1.3? 

In [11]:
# [Answer to ex. 1.1.4 here]

> **Ex. 1.1.5:** Explain why test set performance is often preferred to the training set performance.

In [13]:
# [Answer to ex. 1.1.5 here]

<br>

### Cross validation

We now turn to actually optimizing the model. We will use cross validation to perform the optimization.

> **Ex. 1.1.6:** Explain what the parameter `C` does in logistic regression. How do parameters change when `C` get smaller? Also explain what `penalty='l1'` will do for coefficients.
>> *Hint:* The documentation for scikit learn will tell you everything you need to know about their implementation of logistic regression.

In [15]:
# [Answer to ex. 1.1.6 here]

> **Ex. 1.1.7:** Use cross validation on training set to estimate hyperparameters and then re-estimate model on training set. The set of $\lambda$ to be considered are $\{10^{-4},10^{-2},1,10^{2},10^{4}\}$. Is the model with optimized hyperparameters better than the non-optimized?
>
>> *Hint 1:* This procedure is implemented in `GridSearchCV`. You set `n_jobs=-1` to parallize computation. See Raschka pp. 186-187 for inspiration.
>
>> *Hint 2:* Consider using `np.logspace` for making the set of $\lambda$.m

In [16]:
# [Answer to ex. 1.1.7 here]

> **Ex. 1.1.8:** Apply nested resampling to compute a distribution of test scores with and without optimization. You should use `data_splits` which we defined initially and input all the data.
>
>> *Hint:* You can implement this using your code from Ex. 1.1.6 and combine it with `cross_val_score`. Note that `cv` input should use `data_splits`. See Raschka pp. 188-189 for inspiration. 


In [18]:
# [Answer to ex. 1.1.8 here]

> **Ex. 1.1.9:** Plot the distributions of the optimized vs. the non-optimized. What do you conclude about the performance difference?

In [20]:
# [Answer to ex. 1.1.9 here]

>  **Ex. 1.1.10** Argue how $F_1$ differs from  *area under the curve* (AUC). Comment on their theoretical properties.

### Classification trees and forests

>  **Ex. 1.1.11** Estimate a classification tree on the training data (with default hyperparameters). Evalate both on training and test data by computing the *area under the curve*.
>
>> *Hint:* You can check out code for Ex. 1.1.10 for inspiration. You may also want to look up `roc_auc_score`.

In [22]:
# [Answer to ex. 1.1.11 here]

>  **Ex. 1.1.12** Estimate 50 classification trees and compute the mean prediction. Use the mean prediction to compute the *area under the curve*.
>
>> *Hint:* You may use the code block below and apply it repeatedly.

In [24]:
from sklearn.tree import DecisionTreeClassifier

def train_tree(data_train):
    '''
    Trains a decision tree on a bootstrap 
    of training data.
    '''
    df_ = data_train.sample(frac=1, replace=True)
    ct = DecisionTreeClassifier(class_weight='balanced')
    ct.fit(df_.loc[:,X_vars_b], df_[y_var])
    return ct


In [25]:
# [Answer to ex. 1.1.12 here]

>  **Ex. 1.1.13** Is Random Forest classification different from the procedure of aggregating tree predictions above? If so, explain how.

In [27]:
# [Answer to ex. 1.1.13 here]